# 1 - Import Dependencies

In [1]:
!pip install stable-baselines3

In [2]:
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

C:\Users\dilan\anaconda3\envs\deep-learning\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\dilan\anaconda3\envs\deep-learning\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\dilan\anaconda3\envs\deep-learning\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
C:\Users\dilan\anaconda3\envs\deep-learning\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\dilan\anaconda3\envs\deep-learning\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\dilan\anaconda3\envs\deep-learning\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# 2 - Load Environment

In [3]:
environment_name = 'CartPole-v0'
env = gym.make(environment_name)

In [4]:
environment_name

'CartPole-v0'

'CartPole-v0'

In [5]:
episodes = 5
for episode in range(1, episodes  + 1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episodes:{} Score:{}'.format(episode, score))
env.close()

Episodes:1 Score:33.0
Episodes:1 Score:33.0
Episodes:2 Score:28.0
Episodes:2 Score:28.0
Episodes:3 Score:15.0
Episodes:3 Score:15.0
Episodes:4 Score:25.0
Episodes:4 Score:25.0
Episodes:5 Score:78.0
Episodes:5 Score:78.0


# 3 - Understanding The Environment

In [6]:
env.action_space

Discrete(2)

Discrete(2)

In [7]:
env.action_space.sample()

1

1

In [8]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [9]:
env.observation_space.sample()

array([-3.3716049e+00, -6.7158293e+37,  1.6458732e-01,  5.9956420e+37],
      dtype=float32)

array([-3.3716049e+00, -6.7158293e+37,  1.6458732e-01,  5.9956420e+37],
      dtype=float32)

# 4 - Train an RL Model

In [10]:
log_path = os.path.join('Training', 'Logs')

In [11]:
log_path

'Training\\Logs'

'Training\\Logs'

In [12]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cuda device
Using cuda device


In [13]:
model.learn(total_timesteps=20000)

Logging to Training\Logs\PPO_2
Logging to Training\Logs\PPO_2
-----------------------------
| time/              |      |
|    fps             | 225  |
|    iterations      | 1    |
|    time_elapsed    | 9    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------
| time/              |      |
|    fps             | 225  |
|    iterations      | 1    |
|    time_elapsed    | 9    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 217         |
|    iterations           | 2           |
|    time_elapsed         | 18          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008893569 |
|    clip_fraction        | 0.0976      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 0.00437     |
|    learning_

-----------------------------------------
| time/                   |             |
|    fps                  | 201         |
|    iterations           | 7           |
|    time_elapsed         | 71          |
|    total_timesteps      | 14336       |
| train/                  |             |
|    approx_kl            | 0.007017848 |
|    clip_fraction        | 0.0419      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.583      |
|    explained_variance   | 0.481       |
|    learning_rate        | 0.0003      |
|    loss                 | 17.2        |
|    n_updates            | 60          |
|    policy_gradient_loss | -0.00793    |
|    value_loss           | 58.3        |
-----------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 195          |
|    iterations           | 8            |
|    time_elapsed         | 83           |
|    total_timesteps      | 1

# 5 - Save and Reload Model 

In [14]:
PPO_Path = os.path.join('Training', 'Saved Models', 'PPO_Model_Cartpole')

In [15]:
model.save(PPO_Path)

C:\Users\dilan\anaconda3\envs\deep-learning\lib\site-packages\stable_baselines3\common\save_util.py:276: UserWarning: Path 'Training\Saved Models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")
C:\Users\dilan\anaconda3\envs\deep-learning\lib\site-packages\stable_baselines3\common\save_util.py:276: UserWarning: Path 'Training\Saved Models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [16]:
del model

In [17]:
model = PPO.load(PPO_Path, env=env)

# 6 - Evaluation

In [18]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

C:\Users\dilan\anaconda3\envs\deep-learning\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
C:\Users\dilan\anaconda3\envs\deep-learning\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(200.0, 0.0)

(200.0, 0.0)

In [19]:
env.close()

# 7 - Test Model

In [20]:
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward
    print('Episodes:{} Score:{}'.format(episode, score))
env.close()

Episodes:1 Score:[200.]
Episodes:1 Score:[200.]
Episodes:2 Score:[200.]
Episodes:2 Score:[200.]
Episodes:3 Score:[200.]
Episodes:3 Score:[200.]
Episodes:4 Score:[200.]
Episodes:4 Score:[200.]
Episodes:5 Score:[200.]
Episodes:5 Score:[200.]


# 8 - Viewing Logs in Tensorboard

In [21]:
training_log_path = os.path.join(log_path, 'PPO_2')

In [22]:
!tensorboard --logdir={training_log_path}

^C
^C


# 9 - Adding a callback to the Training Stage

In [34]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [35]:
save_path = os.path.join('Training', 'Saved Models')

In [36]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=True)
eval_callback = EvalCallback(env,
                            callback_on_new_best=stop_callback,
                            eval_freq=10000,
                            best_model_save_path=save_path,
                            verbose=True)

In [37]:
model = PPO('MlpPolicy', env, verbose=True, tensorboard_log=log_path)

Using cuda device
Using cuda device


In [38]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\PPO_5
Logging to Training\Logs\PPO_5
-----------------------------
| time/              |      |
|    fps             | 286  |
|    iterations      | 1    |
|    time_elapsed    | 7    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------
| time/              |      |
|    fps             | 286  |
|    iterations      | 1    |
|    time_elapsed    | 7    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 220         |
|    iterations           | 2           |
|    time_elapsed         | 18          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008958044 |
|    clip_fraction        | 0.109       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.000765   |
|    learning_

# 10 - Changing Policies

In [39]:
net_arch = [dict(pi=[128,128,128,128], vf=[128,128,128,128])]

In [40]:
model = PPO('MlpPolicy', env, verbose=True, tensorboard_log=log_path, policy_kwargs={'net_arch':net_arch})

Using cuda device
Using cuda device


In [41]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\PPO_6
Logging to Training\Logs\PPO_6
-----------------------------
| time/              |      |
|    fps             | 219  |
|    iterations      | 1    |
|    time_elapsed    | 9    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------
| time/              |      |
|    fps             | 219  |
|    iterations      | 1    |
|    time_elapsed    | 9    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 185         |
|    iterations           | 2           |
|    time_elapsed         | 22          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.012937122 |
|    clip_fraction        | 0.17        |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.683      |
|    explained_variance   | -7.51e-05   |
|    learning_

-----------------------------------------
| time/                   |             |
|    fps                  | 162         |
|    iterations           | 7           |
|    time_elapsed         | 88          |
|    total_timesteps      | 14336       |
| train/                  |             |
|    approx_kl            | 0.009131725 |
|    clip_fraction        | 0.0939      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.525      |
|    explained_variance   | 0.84        |
|    learning_rate        | 0.0003      |
|    loss                 | 7.32        |
|    n_updates            | 60          |
|    policy_gradient_loss | -0.00813    |
|    value_loss           | 15.4        |
-----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 162         |
|    iterations           | 7           |
|    time_elapsed         | 88          |
|    total_timesteps      | 14336 

# 10 - Using an Alternate Algorithm

In [42]:
from stable_baselines3 import DQN

In [43]:
model = DQN('MlpPolicy', env, verbose=True, tensorboard_log=log_path)

Using cuda device
Using cuda device


In [44]:
model.learn(total_timesteps=20000)

Logging to Training\Logs\DQN_2
Logging to Training\Logs\DQN_2
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.969    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 3008     |
|    time_elapsed     | 0        |
|    total_timesteps  | 66       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.969    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 3008     |
|    time_elapsed     | 0        |
|    total_timesteps  | 66       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.933    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 3390     |
|    time_elapsed     | 0        |
|    total_timesteps  | 142      |
----------------------------

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.358    |
| time/               |          |
|    episodes         | 56       |
|    fps              | 4083     |
|    time_elapsed     | 0        |
|    total_timesteps  | 1352     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.358    |
| time/               |          |
|    episodes         | 56       |
|    fps              | 4083     |
|    time_elapsed     | 0        |
|    total_timesteps  | 1352     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.297    |
| time/               |          |
|    episodes         | 60       |
|    fps              | 4107     |
|    time_elapsed     | 0        |
|    total_timesteps  | 1479     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 108      |
|    fps              | 4358     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2743     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 112      |
|    fps              | 4402     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2876     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 112      |
|    fps              | 4402     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2876     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 164      |
|    fps              | 4501     |
|    time_elapsed     | 0        |
|    total_timesteps  | 3987     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 164      |
|    fps              | 4501     |
|    time_elapsed     | 0        |
|    total_timesteps  | 3987     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 168      |
|    fps              | 4504     |
|    time_elapsed     | 0        |
|    total_timesteps  | 4079     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 216      |
|    fps              | 4452     |
|    time_elapsed     | 1        |
|    total_timesteps  | 5129     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 220      |
|    fps              | 4418     |
|    time_elapsed     | 1        |
|    total_timesteps  | 5196     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 220      |
|    fps              | 4418     |
|    time_elapsed     | 1        |
|    total_timesteps  | 5196     |
----------------------------------
----------------------------------
| rollout/          

...

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 648      |
|    fps              | 4286     |
|    time_elapsed     | 3        |
|    total_timesteps  | 15019    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 652      |
|    fps              | 4285     |
|    time_elapsed     | 3        |
|    total_timesteps  | 15090    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 652      |
|    fps              | 4285     |
|    time_elapsed     | 3        |
|    total_timesteps  | 15090    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 704      |
|    fps              | 4279     |
|    time_elapsed     | 3        |
|    total_timesteps  | 16182    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 704      |
|    fps              | 4279     |
|    time_elapsed     | 3        |
|    total_timesteps  | 16182    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 708      |
|    fps              | 4281     |
|    time_elapsed     | 3        |
|    total_timesteps  | 16251    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 756      |
|    fps              | 4295     |
|    time_elapsed     | 4        |
|    total_timesteps  | 17213    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 760      |
|    fps              | 4295     |
|    time_elapsed     | 4        |
|    total_timesteps  | 17276    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 760      |
|    fps              | 4295     |
|    time_elapsed     | 4        |
|    total_timesteps  | 17276    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 812      |
|    fps              | 4301     |
|    time_elapsed     | 4        |
|    total_timesteps  | 18356    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 812      |
|    fps              | 4301     |
|    time_elapsed     | 4        |
|    total_timesteps  | 18356    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 816      |
|    fps              | 4303     |
|    time_elapsed     | 4        |
|    total_timesteps  | 18468    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 864      |
|    fps              | 4302     |
|    time_elapsed     | 4        |
|    total_timesteps  | 19492    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 868      |
|    fps              | 4302     |
|    time_elapsed     | 4        |
|    total_timesteps  | 19536    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 868      |
|    fps              | 4302     |
|    time_elapsed     | 4        |
|    total_timesteps  | 19536    |
----------------------------------
----------------------------------
| rollout/          